# RestFull avec Java - JAX-RS

L’objet de cette page est de présenter les service Web REST par la pratique en Java. 

L’idée générale est d’offrir un accès distant à des ressources (identifées par des URL) via une interface commune construite au dessus du protocole http. 

On parlera d'approche RESTful quand l'interface est conforme à une certaine philosophie : Une approche client/serveur classique mais en suivant une logique de navigation dans un hypermedia (HATEOS : Hypermedia As The Engine Of Application State). 

L'API est sans état côté serveur. Cela permettra de l'utiliser de façon transparente même en cas de serveur proxy/cache.

Pour définir un protocole de communication il faut généralement définir (1) un système d'identification (d'adressage) des ressources manipulées, (2) un protocole d'échange, (3) un format d'échange de données éventuellement typées et (4) et (5) un système de gestion des erreurs. 

La logique RESTfull est d'utiliser tout ce que propose HTTP pour écrire une API **en** http.

### L'adressage des ressources
Les ressources de l'application sont identifiées par des URL.

### Le protocole d'échange
Les actions sur les ressources (identifiées par des URLs) sont associées aux verbes standards du protocole HTTP.

  * **GET** : Accès à une ressource identifiée dans l'URL (il peut s'agit d'une collection).      
  * **HEAD** : comme GET mais sans le corps de la requête (seul le header http est retourné).
  * **POST** : création d’une ressource sans donner l'identifiant.
  * **PUT** : mise à jour complète d'une ressource identifiée (voire création en donnant l'identifiant).
  * **DELETE** : suppression d’une ressource

La [RFC 2616](https://www.w3.org/Protocols/rfc2616/rfc2616-sec9.html) (HTTP methods) précise en particulier les nuances entre PUT et POST.

GET, HEAD, PUT et DELETE doivent être idempotentes (un ou plusieurs appel doivent avoir les même effets de bord).

#### Quelques exemples
Obtenir toutes les personnes :
      `GET http://MyServer/MyApp/Persons`
      
Obtenir Une personnes précise par identifiant:
      `GET http://MyServer/MyApp/Persons/1`

Obtenir toutes les personnes avec un filter:
      `GET http://MyServer/MyApp/Persons?ageMin=7&ageMax=16`
      
Supprimer toutes personnes     
      `DELETE http://MyServer/MyApp/Persons`
      
Supprimer une personne     
      `DELETE http://MyServer/MyApp/Persons/1`      
      
### La réprésentation des ressources
Les resources sont générales représentées et échangées à l'aide de langages autodescriptifs comme XML ou JSON.
```xml
<?xml version='1.0'?>
<person id='1'>
    <lastname>Doe</lastname>
    <firstname>John</firstname>
</person>
```

```json
{
  "person": {
    "-id": "1",
    "lastname": "Doe",
    "firstname": "John"
  }
}
```
Pour indiquer ce que l'on envoie (Content-Type) ou ce que l'on attend (Accept) comme type de données il est possible d'utiliser les types Internet Media Types appelés à l'origine MIME (Multipurpose Internet Mail Extensions). Il s'agit entre autres d'une liste standard de formats et de sous-formats d'échange de données (text/plain, text/xml, application/json, ...)

### La gestion des erreurs
Le protocole http défini un ensemble de [codes de retours](https://tools.ietf.org/html/rfc2616#section-10).
Un code est indiqué dans l'entête de la réponse (404 non trouvé, 500 erreur interne, 2XX ok, ...).
Il est donc possible d'utiliser ces codes standards comme code retour.

### Un échange REST = un échange HTTP
Les échange d'une API REST correspondent donc exactement à un échange http :

  * Une requête http composée d'un verbe, d'une URI, de la version de http utilisée, d'un en-tête (un liste de couples `nom:valeur` par exemple `Content-Type: text/xml`) et un corps éventuellement vide (les données envoyées).  
  * Une réponse http composée 

#### Quelques exemples 
Création d'une personne :
```
POST http://MyServer:8080/MyApp/Persons/
Host: MyServer:8080
Content-Type: application/json; charset=utf-8
Content-Length: 36
{"lastname": "Doe",
 "firstname": "John"}
```

Modification d'une personne (`id` dans l'URL):
```
PUT http://MyServer:8080/MyApp/Persons/1
Host: MyServer:8080
Content-Type: application/json; charset=utf-8
Content-Length: 12
{"age":"18"}
```

Les liens entre les resources peuvent être réprésentés à l'aide d'URLs.
Pour simplifier l'usage de l'API ces liens peuvent être fournis dans l'entête d'une réponse http. 
```
first-person: http://MyServer:8080/MyApp/Persons/1
next-person: http://MyServer:8080/MyApp/Persons/10
previous-person: http://MyServer:8080/MyApp/Persons/8
last-person: http://MyServer:8080/MyApp/Persons/90
```

La requête interactive suivante utilise la commande (curl)[https://curl.se/] pour soumettre une requête rest à l'API de github pour consulter les données du repository https://github.com/ebpro/sample-jaxrs. 

In [ ]:
%%shell 
curl -s -D - https://api.github.com/repos/ebpro/sample-jaxrs

A partir de ce résultat complétez la requête ci-dessous pour obtenir les branches de ce repository.

In [2]:
%%shell
curl -s -D - https://api.github.com/repos/ebpro/sample-jaxrs

EvalException: Undefined cell magic 'shell'

## Pratique avec Jersey et Grizzly

Java propose un standard appelé [JAX-RS](https://jakarta.ee/specifications/restful-ws/3.0/jakarta-restful-ws-spec-3.0.html) pour construire efficacement des serveurs REST. 
[Jersey](https://eclipse-ee4j.github.io/jersey/download.html) est l'implantation de référence de JAX-RS. 

Dans un premier temps nous allons étudier une application simple qui s'appuie une serveur web en Java [Grizzly](https://javaee.github.io/grizzly/).

L’archetype maven suivant permet de créer un projet de base dans le répertoire `/src/jaxrs/myresource`. 

Une fois la commande terminée (un nombre a remplacé \[*\] cela peut prendre un peu de temps), le code source peut être consulté en utilisant l'IDE [Visual Studio Code](/code-server) embarqué en cliquant sur le lien précédent ou depuis les Launchers.

In [1]:
%%shell
mkdir -p /src/jaxrs/
cd /src/jaxrs/
rm -rf /src/jaxrs/myresource

mvn archetype:generate -B --no-transfer-progress --quiet \
  -DarchetypeGroupId=org.glassfish.jersey.archetypes \
  -DarchetypeArtifactId=jersey-quickstart-grizzly2 \
  -DarchetypeVersion=3.0.0 \
  -DgroupId=fr.univtln.bruno.demos.jaxrs \
  -DartifactId=myresource

<div class="alert alert-block alert-info">
Ouvrez un terminal dans jupyterlab (+ en haut à gauche puis terminal) et copier/coller la commande suivante pour compilier/exécuter l’application et ainsi démarrer le serveur REST.
</div>

```shell
cd /src/jaxrs/myresource
mvn package && mvn exec:java
```

Il est possible d’accéder à la ressource en ligne de commande. 

Regarder les options de la commande curl pour réaliser des requêtes HTTP. 

Des extensions pour les navigateurs web existent aussi comme postman pour Chrome ou RestClient pour firefox (attention les serveurs lancé dans jupyter ne sont pas accessible de l'extérieur).

La commande suivante exécute une requête GET sur l'URL d'une ressource et affiche le résultat en-tête compris.

In [4]:
%%shell
curl -s -D - --get http://localhost:8080/myapp/myresource

EvalException: Undefined cell magic 'shell'

En théorie l'URL `/myapp/application.wadl` permet d'accéder à une description standard de l'API dans un format XML appelé [WADL](https://www.w3.org/Submission/wadl/). 

Cependant, pour cela l'application nécessite une implantation du standard pour traiter les documents XML appelé [JAXB](https://eclipse-ee4j.github.io/jaxb-ri/) qui n'est plus fournie en standard dans le JDK depuis la version 8 (nous utilisons ici la version 11).  C'est pourquoi une erreur 404 est reçue.

In [5]:
%%shell
curl -s -D - --get http://localhost:8080/myapp/application.wadl

EvalException: Undefined cell magic 'shell'

La classe `fr.univtln.bruno.demos.jaxrs.Main` démarre le serveur et paramètre les package où chercher des ressources (cf. commentaires).

La classe `fr.univtln.bruno.demos.jaxrs.MyResource` présente le fonctionnement minimal d'une ressource. La classe est annotée avec `@Path(your_path)` pour indiquer le chemin à ajouter à l’URL correspondant à cette ressource.

Les méthodes sont annotées avec @POST, @GET, @PUT, @DELETE pour indiquer le type de verbe http associée. Les méthodes associées (dont GET) peuvent être annotées avec `@Produces` qui indique le type MIME dans lequel le résultat doit être fourni : `@Produces("text/plain")`, `@Produces("application/json")`, … Il possible d’indiquer plusieurs types avec `@Produces({"application/json", "application/xml"})`. Il existe aussi des constantes équivalentes `MediaType.TEXT_PLAIN`. Une valeur par défaut de `@Produces` peut être indiquée en annotant la classe.

Le client peut indiquer le type demandé avec la valeur `Content-Type: ` de l'entête de la requête.

Les méthodes qui nécessitent des paramètres sont annotées avec `@Consumes(type[, type, …])` pour indiquer les types mime supportés. 

L’annotation @PathParam permet d’injecter les valeurs provenant des URL comme des paramètres. 

## Un exemple avancé

Le projet dans l’entrepôt git https://github.com/ebpro/sample-jaxrs donne un exemple un peu plus complet.

La commande suivante clone le projet :

In [6]:
%%shell 
cd /src/jaxrs/
rm -rf /src/jaxrs/sample-jaxrs
git clone --branch develop https://github.com/ebpro/sample-jaxrs.git

EvalException: Undefined cell magic 'shell'

Ce projet peut aussi être édité dans [Visual Studio Code](/code-server). 

Vous pouvez exécuter la commande suivante depuis un terminal jupyterlab qui utilise le but maven `verify`.
Elle compile, exécute les tests unitaires, package et exécute les tests d'intégration (en lançant le serveur REST et en exécutant de vraies requêtes).

Le but exec:java lances ensuite le serveur (vous pourrez l'arrêter avec ctrl-c dans le terminal). 

<div class="alert alert-block alert-info">
Ouvrez un autre terminal JupyterLab pour exécuter la commande suivante.
</div>

```shell 
cd /src/jaxrs/sample-jaxrs && \
    mvn clean verify &&
    mvn exec:java
```

La classe `fr.univtln.bruno.samples.jaxrs.server.BiblioServer` paramètre Jersey, démarre Grizzly et attend un CTRL-C pour arrêter le serveur. 

La classe `fr.univtln.bruno.samples.jaxrs.model.BiblioModel` définit le modèle de donnée (Une bibliothèque qui est une facade pour gérer des Auteurs.)

La classe `fr.univtln.bruno.samples.jaxrs.resources.BiblioResource` définit une resource facade.


La méthode `sayHello()` reprend l'exemple précédent.

In [7]:
%%shell
echo -e "\033[0;31mGet a Hello message\033[0m"
curl -s -i http://localhost:9998/myapp/biblio

EvalException: Undefined cell magic 'shell'

### Chemins et Verbes
La méthode `init()` est un simple PUT sans paramètre qui initialise la bibliothèque avec deux auteurs.

In [8]:
%%shell
echo "\033[0;31mInit the database with two authors\033[0m"
curl -s -i -X PUT "http://localhost:9998/myapp/biblio/init"

EvalException: Undefined cell magic 'shell'

## Les paramètres simples et la sérialisation du retour
Retourner un auteur par id. La méthode `getAuteur(@PathParam("id") final long id)` s'exécute lors d'un `GET` sur un chemin de forme `@Path("auteurs/{id}")`. `id` est est un pas de chemin quelconque qui sera extrait, converti en long et injecté grâce à @PathParam dans le paramètre id de la fonction. Il est possible d'indiquer une expression régulière pour contraindre la forme du pas par exemple `@Path("auteurs/{id: [0-9]+}")`. Le Produces indique que du XML ou du JSON peuvent être produits. La requête suivante demande du JSON. 

In [9]:
%%shell
echo "\033[0;31mGet author 1 in JSON\033[0m"
curl -s -i -H "Accept: application/json"  \
  http://localhost:9998/myapp/biblio/auteurs/1

EvalException: Undefined cell magic 'shell'

La requête suivante reprend la précédente et demande du XML. JAX-RS va chercher automatiquement des classes (MessageBodyWriter et Reader) pour créer le bon format. Ces classes peuvent construites explicitement mais des extensions peuvent être ajoutées pour produire les types classiques par annotations des entités (cf. le pom.xml et  les annotations de la classe `BiblioModel.Auteur`) : jersey-media-jaxb pour XML et jersey-media-json-jackson pour JSON. Jackson n'est pas l'implantatation pas défaut mais elle est plus efficace et plus configurable. 

In [10]:
%%shell
echo "\033[0;31mGet author 2 in XML\033[0m"
curl -s -i -H "Accept: text/xml"  \
  http://localhost:9998/myapp/biblio/auteurs/2

EvalException: Undefined cell magic 'shell'

Les collections classiques sont supportés. Notez qu'ici les [collections eclipse](https://www.eclipse.org/collections/) sont utilisées en particulier celles pour les primitifs et qu'elles sont supportées par Jackson.

In [11]:
%%shell
echo "\033[0;31mGet authors in JSON\033[0m"
curl -s -i -H "Accept: application/json"  \
  http://localhost:9998/myapp/biblio/auteurs

EvalException: Undefined cell magic 'shell'

D'une façon similaire les annotations `@HeaderParam` et `@QueryParam` permettent d'extraire des valeurs de l'entête ou des paramètres de la requête http :

```shell
```

### La sérialisation des données

Depuis Java 9 il est nécessaire d'ajouter les dépendances suivante pour traiter des données XML : 

```XML
<dependency>
  <groupId>jakarta.xml.bind</groupId> 
  <artifactId>jakarta.xml.bind-api</artifactId>
</dependency>
<dependency>
  <groupId>org.glassfish.jaxb</groupId>
  <artifactId>jaxb-runtime</artifactId>
</dependency>
```

La définition des formats de données XML se fait par annotation des entités en utilisant le standard JAXB (Java Architecture for XML Binding) : @XmlElement, @XmlType, @XmlAttribute, @XmlTransient, @XmlValue, ...

Le standard officiel pour JSON est maintenant JSON-B (Java API for JSON Binding). Cependant, des fonctionnalités importantes sont encore manquantes ,ous utiliserons donc une autre librairie : Jackson https://github.com/FasterXML/jackson (cf. pom.xml).

### Les paramètres complexes dans les corps de requêtes 
Les conversions sont aussi automatiques dans l'autre sens quand des données sont envoyées dans le corps d'une requête ici du JSON dans un POST.
L'annotation @Consummes de la méthode addAuteur() indique ce qui est possible et l'entete Content-Type ce qui est envoyé. Noter que l'Id n'est pas indiqué mais que l'entité complète est retournée. 

In [12]:
%%shell
echo "\033[0;31mAdds an author\033[0m"
curl -s -i -H "Accept: application/json"  \
  -H "Content-type: application/json"  \
  -X POST \
  -d '{"nom":"John","prenom":"Smith","biographie":"My life"}' \
  http://localhost:9998/myapp/biblio/auteurs/

EvalException: Undefined cell magic 'shell'

La méthode `updateAuteur` est appelée par un PUT mais avec une resource précise (indiquée dans l'URL) à mettre à jour.

In [13]:
%%shell
echo "\033[0;31mFully update an author\033[0m"
curl -s -i -H "Accept: application/json"  \
  -H "Content-type: application/json"  \
  -X PUT \
  -d '{"nom":"Martin","prenom":"Jean","biographie":"ma vie"}' \
  http://localhost:9998/myapp/biblio/auteurs/1

EvalException: Undefined cell magic 'shell'

### Les exceptions
Le traitement des exceptions peut être simplifié en utilisant des mappers (cf. package exceptions et mapper) qui s'appliquent automatiquement lors qu'une exception est émise. Dans ce cas un objet Response est construit manuellement pour contrôler le détail du corps et de l'entête. 

In [14]:
%%shell
echo "\033[0;31mIf a resource doesn't exist an exception is raised, and the 404 http status code is returned\033[0m"
curl -s -i -H "Accept: application/json"  \
  http://localhost:9998/myapp/biblio/auteurs/1000

EvalException: Undefined cell magic 'shell'

D'une façon générale une classe annotée par `@Provider` ajouter des fonctions (traitement des exceptions, conversion des données, ...).

### La suppression
La suppression des requêtes se fait avec les approches précédentes.

In [15]:
%%shell
echo "\033[0;31mRemoves one author\033[0m"
    curl -s -i -H "Accept: application/json"  \
curl -s -i -X DELETE \
    http://localhost:9998/myapp/biblio/auteurs/1

EvalException: Undefined cell magic 'shell'

In [16]:
%%shell
echo "\033[0;31mRemoves all authors\033[0m"
curl -s -i -X DELETE \
    http://localhost:9998/myapp/biblio/auteurs

EvalException: Undefined cell magic 'shell'

In [17]:
%%shell
echo "\033[0;31mInit the database with two authors\033[0m"
curl -s -i -X PUT http://localhost:9998/myapp/biblio/init

EvalException: Undefined cell magic 'shell'

### Définition de l'API
Si JAXB est dans le classpath (cf. pom.xml) la génération d'une description WADL est automatique.

In [18]:
%%shell
curl -s -D - --get http://localhost:9998/myapp/application.wadl

EvalException: Undefined cell magic 'shell'

### Une API Cliente
La classe `fr.univtln.bruno.samples.jaxrs.client.BiblioClient` définit un client qui utilise l'API fluent cliente pour construire des requêtes REST en Java.

In [19]:
%%shell 
cd /src/jaxrs/sample-jaxrs
mvn  -Dmain.class="fr.univtln.bruno.samples.jaxrs.client.BiblioClient" exec:java

EvalException: Undefined cell magic 'shell'

## Pour aller plus loin
L'un des grandes force de REST est sa capacité à fonctionne avec tout les langages de programmation y compris les scripts. 

<div class="alert alert-block alert-info">
  * Commencez à ajouter une API REST à un programme que vous avez déjà fait. 
  * Ecrivez un client REST pour cette API dans un autre langage de programmation comme python.
</div>

La sécurité est apportée par http : pour protéger les communications il suffit d'activer SSL dans le serveur ou d'accéder au serveur REST via un reverse proxy sécurisé. 

L'authentification est cruciale, elle sera étudiée en détail en s'appuyant sur les JSON Web Token (https://jwt.io/).